### Lets do some feature Engineering that includes create dummy variabble on categorical data and scaling on numerical data

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [58]:
df = pd.read_csv("LendingClub_cleaned.csv")

In [59]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE,...,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE,...,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
2,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,10+ years,MORTGAGE,...,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,N,Cash,N
3,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE,...,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N
4,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,4 years,RENT,...,100.0,0.0,0.0,16900.0,12798.0,9400.0,4000.0,N,Cash,N


In [89]:
df['loan_status'].value_counts()

Fully Paid            880570
Current               734292
Charged Off           210041
Late (31-120 days)     17051
In Grace Period         6949
Late (16-30 days)       3391
Default                   30
Name: loan_status, dtype: int64

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852324 entries, 0 to 1852323
Data columns (total 88 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   loan_amnt                   float64
 1   funded_amnt                 float64
 2   funded_amnt_inv             float64
 3   term                        object 
 4   int_rate                    float64
 5   installment                 float64
 6   grade                       object 
 7   sub_grade                   object 
 8   emp_length                  object 
 9   home_ownership              object 
 10  annual_inc                  float64
 11  verification_status         object 
 12  issue_d                     object 
 13  loan_status                 object 
 14  pymnt_plan                  object 
 15  purpose                     object 
 16  title                       object 
 17  addr_state                  object 
 18  dti                         float64
 19  delinq_2yrs          

In [61]:
dfo = df.select_dtypes(include = "object")

In [62]:
dfo.head()

,term,grade,sub_grade,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,disbursement_method,debt_settlement_flag
0,36 months,C,C4,10+ years,MORTGAGE,Not Verified,Dec-2015,Fully Paid,n,debt_consolidation,Debt consolidation,PA,Aug-2003,w,Jan-2019,Mar-2019,Individual,N,Cash,N
1,36 months,C,C1,10+ years,MORTGAGE,Not Verified,Dec-2015,Fully Paid,n,small_business,Business,SD,Dec-1999,w,Jun-2016,Mar-2019,Individual,N,Cash,N
2,60 months,C,C5,10+ years,MORTGAGE,Source Verified,Dec-2015,Current,n,debt_consolidation,Debt consolidation,NJ,Sep-2008,w,Feb-2019,Mar-2019,Individual,N,Cash,N
3,60 months,F,F1,3 years,MORTGAGE,Source Verified,Dec-2015,Fully Paid,n,major_purchase,Major purchase,PA,Jun-1998,w,Jul-2016,Mar-2018,Individual,N,Cash,N
4,36 months,C,C3,4 years,RENT,Source Verified,Dec-2015,Fully Paid,n,debt_consolidation,Debt consolidation,GA,Oct-1987,w,May-2017,May-2017,Individual,N,Cash,N


In [63]:
dfo.term.value_counts()

 36 months    1298260
 60 months     554064
Name: term, dtype: int64

In [64]:
dfo.emp_length.value_counts()

10+ years    664819
2 years      178867
3 years      158672
< 1 year     156871
1 year       128932
5 years      121512
4 years      119073
6 years       89784
7 years       81737
8 years       81427
9 years       70630
Name: emp_length, dtype: int64

### Let's decide to drop some of the categorical features that may not be useful for our purposes:

In [65]:
drop_list = ["term", "sub_grade", "issue_d", "title", "addr_state", "earliest_cr_line", "last_pymnt_d", "initial_list_status", \
            "last_credit_pull_d", "disbursement_method", "debt_settlement_flag"]

In [66]:
df1 = df.drop(drop_list, axis = 1, )

In [67]:
df1.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag
0,3600.0,3600.0,3600.0,13.99,123.03,C,10+ years,MORTGAGE,55000.0,Not Verified,...,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N
1,24700.0,24700.0,24700.0,11.99,820.28,C,10+ years,MORTGAGE,65000.0,Not Verified,...,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N
2,35000.0,35000.0,35000.0,14.85,829.90,C,10+ years,MORTGAGE,110000.0,Source Verified,...,1.0,100.0,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,N
3,10400.0,10400.0,10400.0,22.45,289.91,F,3 years,MORTGAGE,104433.0,Source Verified,...,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N
4,11950.0,11950.0,11950.0,13.44,405.18,C,4 years,RENT,34000.0,Source Verified,...,0.0,100.0,100.0,0.0,0.0,16900.0,12798.0,9400.0,4000.0,N


## Let's drop "Loan_status" colum from dataframe and save it as a y variable

In [91]:
y=df1["loan_status"]

In [96]:
df1.drop(["loan_status"],axis =1, inplace = True)

In [98]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = df1.copy()
numeric_cols = list(df_scaled.dtypes[df_scaled.dtypes != 'object'].index)
df_scaled.loc[:,numeric_cols] = scaler.fit_transform(df_scaled.loc[:,numeric_cols])

In [99]:
df_scaled.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag
0,-1.278737,-1.278735,-1.278469,0.197343,-1.238047,C,10+ years,MORTGAGE,-0.229659,Not Verified,...,0.499240,-1.950105,-1.168417,-0.34231,-0.124905,-0.036211,-0.904895,-0.929861,-0.719997,N
1,1.002997,1.002998,1.003790,-0.214999,1.361097,C,10+ years,MORTGAGE,-0.140051,Not Verified,...,-0.048723,0.357075,-0.955033,-0.34231,-0.124905,0.713824,-0.281909,2.429453,-0.480902,N
2,2.116829,2.116830,2.117878,0.374650,1.396957,C,10+ years,MORTGAGE,0.263187,Source Verified,...,-0.596686,0.649693,-1.168417,-0.34231,-0.124905,1.084509,-0.031548,1.695559,-0.626704,N
3,-0.543391,-0.543390,-0.542954,1.941550,-0.615967,F,3 years,MORTGAGE,0.213302,Source Verified,...,1.047203,0.267039,0.494316,-0.34231,-0.124905,1.406413,0.823381,-0.147914,0.906255,N
4,-0.375776,-0.375774,-0.375300,0.083949,-0.186275,C,4 years,RENT,-0.417837,Source Verified,...,-1.144649,0.649693,1.602804,-0.34231,-0.124905,-0.925164,-0.805701,-0.624071,-0.932871,N


In [100]:
df_scaled_dum = pd.get_dummies(df_scaled)

In [101]:
df_scaled_dum.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,...,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y
0,-1.278737,-1.278735,-1.278469,0.197343,-1.238047,-0.229659,-1.128452,-0.357410,-0.723164,-0.723152,...,0,0,0,0,0,0,1,0,1,0
1,1.002997,1.002998,1.003790,-0.214999,1.361097,-0.140051,-0.250273,0.785254,0.495685,0.495669,...,0,0,0,1,0,0,1,0,1,0
2,2.116829,2.116830,2.117878,0.374650,1.396957,0.263187,-0.163753,-0.357410,2.628671,2.628606,...,0,0,0,0,0,0,1,0,1,0
3,-0.543391,-0.543390,-0.542954,1.941550,-0.615967,0.213302,0.555230,0.785254,-0.113739,-0.113742,...,0,0,0,0,0,0,1,0,1,0
4,-0.375776,-0.375774,-0.375300,0.083949,-0.186275,-0.417837,-0.757281,-0.357410,-0.266095,-0.266094,...,0,0,0,0,0,0,1,0,1,0


In [108]:
X = df_scaled_dum

### It seems like the data is normalized and dummy variable is created for categorical features. Now, Lets, split the data into training and testing sets and get ready to apply machine learning algorithms.

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [112]:
print("Shape of X_train:", np.shape(X_train))
print("Shape of y_train:", np.shape(y_train))
print()
print("Shape of X_test:", np.shape(X_test))
print("Shape of y_test:", np.shape(y_test))

Shape of X_train: (1481859, 115)
Shape of y_train: (1481859,)

Shape of X_test: (370465, 115)
Shape of y_test: (370465,)
